# Univariate Gaussian Mixture – Num Processors

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

import multiprocessing as mp

from ebc.sequential.iterative_with_convexification import SensitivityBasedFW
from ebc.gaussian import gaussain_univaraite_log_likelihood

from splitting import split_based_on_ML, split_randomly, distribute
from parallelization import parallelize

from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings(action = "ignore", category = FutureWarning)

import pickle

## Data Generation

In [ ]:
np.random.seed(123)

full_means = [6, 8, 10]
full_sigmas = [0.8, 0.5, 0.2]
probs = [0.2, 0.5, 0.3]

mixture_inds = np.random.choice([0, 1, 2], size = 3000, replace = True, p = probs)
mixture = []

for i in mixture_inds:
    if i == 0:
        mixture.append(np.random.normal(6, 0.8))
    if i == 1:
        mixture.append(np.random.normal(8, 0.5))
    if i == 2:
        mixture.append(np.random.normal(10, 0.2))

mixture = np.array(mixture).reshape(-1, 1)

plt.hist(mixture, bins = 100);

## Log-likelihood Definition

In [ ]:
def log_likelihood(params, X, y, weights):
    '''
    Returns:
    ----------
    log_lik: np.ndarray(shape = X.shape[0])
    '''
    probs = params[:3]
    mu = params[3:6].reshape(-1, 1)
    sigma = params[6:9]

    ll = 0
    for i in range(3):
        ll += probs[i] * gaussain_univaraite_log_likelihood(X, mu[i], sigma[i])
    
    return ll

def summed_log_likelihood(params, X, y, weights):
    return log_likelihood(params, X, y, weights).sum()

def negative_summed_log_likelihood(params, X, y, weights):
    return -summed_log_likelihood(params, X, y, weights)

def grad_log_likelihood(params, X, y, weights):
    return None

def log_posterior(params, X, y, weights):
    return weights.T @ log_likelihood(params, X, y, weights)

In [ ]:
# https://stats.stackexchange.com/questions/7440/kl-divergence-between-two-univariate-gaussians
# https://math.stackexchange.com/questions/2614267/can-we-solve-kl-divergence-between-gaussian-mixtures-by-thinking-conditional-cas

def mixture_kl_element(mu0, sigma0, mu1, sigma1, w):
    KL = np.log(sigma1 / sigma0) + (sigma1 ** 2 + (mu1 - mu0) ** 2) / (2 * sigma1 ** 2) - 1/2
    return KL * w

In [ ]:
num_proc = [2, 3, 4, 5, 6]

## Testing

In [ ]:
x = mixture

# Parallel
fkl_parallel = []
bkl_parallel = []
time_parallel = []

for k in range(20):
      np.random.seed(120 + k)
      fkl_parallel_k = []
      bkl_parallel_k = []
      time_parallel_k = []

      for i in num_proc:

            print(f"{k}: {i}")
            
            na = {"log_likelihood": log_likelihood,
                  "log_likelihood_start_value": np.ones(9),
                  "S": int(0.7 / i * len(x) * 0.5),
                  "log_likelihood_gradient": None,
                  "approx": "MCMC",
                  "MCMC_subs_size": int(0.7 / i * len(x)),
                  "log_posterior": log_posterior,
                  "log_posterior_start_value": np.ones(9)}
            
            fkl_parallel_i = []
            bkl_parallel_i = []
            time_parallel_i = []
            for ind, strat in enumerate([split_randomly, split_based_on_ML]):
                  start = time.time()

                  # Step 1: distribute
                  if ind == 0:
                        full_inds = strat(x, i)
                  elif ind == 1:
                        gm = GaussianMixture(3)
                        gm.fit(mixture)
                        # Get probability estimates
                        params = np.hstack((gm.weights_.flatten(), gm.means_.flatten(), gm.covariances_.flatten()))
                        log_liks = log_likelihood(params, mixture, None, None)
                        probs = np.abs(log_liks) / np.sum(np.abs(log_liks))
                        probs = probs.flatten()
                        full_inds = distribute(probs, i)

                  # Step 2: run
                  w = parallelize(alg = SensitivityBasedFW, x = x, k = int(300 / i), norm = "2", na = na, distributed_indices = full_inds,
                                  num_proc = i)

                  time_parallel_i.append(time.time() - start)

                  # Calculate posterior approximation
                  gm = GaussianMixture(3)
                  gm.fit((w * mixture).reshape((-1, 1)))
                  means = gm.means_.flatten()
                  sigmas = gm.covariances_.flatten()
                  ps = gm.weights_.flatten()

                  fkl = 0
                  bkl = 0
                  for j in range(3):
                        fkl += mixture_kl_element(full_means[j], full_sigmas[j], means[j], sigmas[j], ps[j])
                        bkl += mixture_kl_element(means[j], sigmas[j], full_means[j], full_sigmas[j], ps[j])

                  fkl_parallel_i.append(fkl)
                  bkl_parallel_i.append(bkl)

            fkl_parallel_k.append(fkl_parallel_i)
            bkl_parallel_k.append(bkl_parallel_i)
            time_parallel_k.append(time_parallel_i)

      fkl_parallel.append(fkl_parallel_k)
      bkl_parallel.append(bkl_parallel_k)
      time_parallel.append(time_parallel_k)

print(f"FKL: {fkl_parallel}")
print(f"BKL: {bkl_parallel}")
print(f"Time: {time_parallel}")

In [ ]:
fkl_parallel = np.array(fkl_parallel)
bkl_parallel = np.array(bkl_parallel)
time_parallel = np.array(time_parallel)

In [ ]:
data = {
    'fkl_parallel': fkl_parallel,
    'bkl_parallel': bkl_parallel,
    'time_parallel': time_parallel
}

In [ ]:
with open("data/univariate_gaussian_mixture_proc.pickle") as file:
    pickle.dump(data, file, protocol = pickle.HIGHEST_PROTOCOL)

## Plot

In [ ]:
with open("data/univariate_gaussian_mixture.pickle", "rb") as file:
    data = pickle.load(file)

fkl_sequential = data['fkl_sequential'][:, -1]
bkl_sequential = data['bkl_sequential'][:, -1]
time_sequential = data['time_sequential'][:, -1]

with open("data/univariate_gaussian_mixture_proc.pickle") as file:
    data = pickle.load(file)

fkl_parallel = data['fkl_parallel']
bkl_parallel = data['bkl_parallel']
time_parallel = data['time_parallel']

In [ ]:
klsym_parallel = (fkl_parallel + bkl_parallel) / 2
klsym_sequential = [np.median((fkl_sequential + bkl_sequential) / 2)] * len(num_proc)
time_sequential = [np.median(time_sequential)] * len(num_proc)

In [ ]:
plt.rcParams.update({'font.size': 22})
fig = plt.figure(figsize = (20, 7))

ax12 = fig.add_subplot(121)

ax13 = fig.add_subplot(222)
ax14 = fig.add_subplot(224, sharex = ax13)

ax12.plot(num_proc, np.log(klsym_sequential), label = 'Sequential', 
          linestyle = "solid", linewidth = 2, color = 'black')
ax12.plot(num_proc, np.log(np.median(klsym_parallel, axis = 0)[:, 0]), label = 'Random split',
          linestyle = "dashed", linewidth = 2, color = 'dimgray')
ax12.plot(num_proc, np.log(np.median(klsym_parallel, axis = 0)[:, 1]), label = 'ML split',
          linestyle = "solid", marker = "o", linewidth = 2, color = 'maroon')

ax13.spines['bottom'].set_visible(False)
ax13.xaxis.tick_top()
ax13.tick_params(labeltop = False)
ax14.spines['top'].set_visible(False)
ax14.ticklabel_format(useOffset=False)

ax12.set_xlabel("Processors")
ax14.set_xlabel("Processors")

fig.text(0.08, 0.5, 'Log KL', va='center', rotation='vertical')
fig.text(0.494, 0.5, 'Seconds', va='center', rotation='vertical')

d = .015

kwargs = dict(transform=ax13.transAxes, color='k', clip_on=False)
ax13.plot((-d, +d), (-d, +d), **kwargs)
ax13.plot((1 - d, 1 + d), (-d, +d), **kwargs)

kwargs.update(transform=ax14.transAxes)
ax14.plot((-d, +d), (1 - d, 1 + d), **kwargs)
ax14.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)

fig.legend()
fig.suptitle('Univariate Gaussian Mixture (k = 300)')

ax13.plot(num_proc, time_sequential, label = 'Sequential',
          linestyle = "solid", linewidth = 2, color = 'black')
ax14.plot(num_proc, np.median(time_parallel, axis = 0)[:, 0], label = 'Random split',
          linestyle = "dashed", linewidth = 2, color = 'dimgray')
ax14.plot(num_proc, np.median(time_parallel, axis = 0)[:, 1], label = 'ML split',
          linestyle = "solid", marker = "o", linewidth = 2, color = 'maroon')

ax12.grid()
ax13.grid()
ax14.grid()

plt.savefig("plots/univariate_gaussian_mixture_proc.eps")

plt.show()